# Classification using Pytorch and Torchtext

- bidirectional LSTM on some news
- some labeled data on news....

- Torchtext to help us numericalize and load some data
- Torchtext is backed by PyTorch....so naturally torchtext is quite good
- Torchtext is not meant to replace spacy....spacy is still like better in general
  
- PyTorch to help us make some neural network

In [5]:
import torch, torchdata, torchtext
from torch import nn

import time

#1. puffer - it's outdated....
#2. spend some money - 300 baht get collab pro

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#reproducibility 
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cpu


In [6]:
torch.__version__

'1.13.0'

In [7]:
torchtext.__version__

'0.14.0'

In [8]:
torchdata.__version__

'0.5.1'

In [10]:
# torch.cuda.get_device_name(0)

## 1. Load the dataset

Make our life easy by using some ready-to-be-used dataset by torchtext

- in your assignment, i will ask you to use penn treebank.....

In [12]:
#if you are using puffer
import os
os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

from torchtext.datasets import AG_NEWS
train, test = AG_NEWS()

In [13]:
train  #a new object by torchdata.....streaming data (yield ....)

ShardingFilterIterDataPipe

## 2. EDA - exploratory data analysis
- check common words
- look at some random sample....how it looks, so that we can design proper neural network
- visualize statistics

In [17]:
next(iter(train))  #generator
# (“World”, “Sports”, “Business”, “Sci/Tech”)
#  1,        2,        3,          4

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [23]:
list(iter(train))[100]  #generator
# (“World”, “Sports”, “Business”, “Sci/Tech”)
#  1,        2,        3,          4

(4,
 'Comets, Asteroids and Planets around a Nearby Star (SPACE.com) SPACE.com - A nearby star thought to harbor comets and asteroids now appears to be home to planets, too. The presumed worlds are smaller than Jupiter and could be as tiny as Pluto, new observations suggest.')

In [19]:
set([y for y, x in list(iter(train))])

{1, 2, 3, 4}

In [25]:
train_size = len(list(iter(train)))
train_size

120000

In [26]:
train

ShardingFilterIterDataPipe

In [29]:
#i gonna cheat a little bit, not gonna use all...my fans will work too hard....
too_much, train, valid = train.random_split(total_length=train_size, 
                                            weights = {"too_much": 0.7, 
                                                       "smaller_train": 0.2,
                                                       "valid": 0.1},
                                            seed = SEED)

In [30]:
train_size = len(list(iter(train)))
val_size   = len(list(iter(valid)))
test_size  = len(list(iter(test)))

In [31]:
train_size, val_size, test_size

(24000, 12000, 7600)